In [1]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
from sklearn.linear_model import LinearRegression
from scipy.stats import spearmanr

In [2]:
!wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/course_library/andrea_models.py
!wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/course_library/feature_engineering.py

import feature_engineering

--2024-11-02 14:35:32--  https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/course_library/andrea_models.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3190 (3.1K) [text/plain]
Saving to: ‘andrea_models.py’

andrea_models.py    100%[===================>]   3.12K  --.-KB/s    in 0s      

2024-11-02 14:35:32 (53.9 MB/s) - ‘andrea_models.py’ saved [3190/3190]

--2024-11-02 14:35:32--  https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/course_library/feature_engineering.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected

In [5]:
data_path = './data/'
df_X_train = pd.read_csv(data_path+'X_train.csv').drop(['COUNTRY'], axis=1).fillna(0) # given by the benchmark
df_X_test = pd.read_csv(data_path+'X_test.csv')
df_y_train = pd.read_csv(data_path+'Y_train.csv')

In [12]:
df_X_train

,ID,DAY_ID,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,FR_NET_IMPORT,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
0,1054,206,0.210099,-0.427458,-0.606523,0.606523,0.000000,0.692860,0.000000,-0.692860,...,-0.444661,-0.172680,-0.556356,-0.790823,-0.283160,-1.069070,-0.063404,0.339041,0.124552,-0.002445
1,2049,501,-0.022399,-1.003452,-0.022063,0.022063,-0.573520,-1.130838,0.573520,1.130838,...,-1.183194,-1.240300,-0.770457,1.522331,0.828412,0.437419,1.831241,-0.659091,0.047114,-0.490365
2,1924,687,1.395035,1.978665,1.021305,-1.021305,-0.622021,-1.682587,0.622021,1.682587,...,1.947273,-0.480700,-0.313338,0.431134,0.487608,0.684884,0.114836,0.535974,0.743338,0.204952
3,297,720,-0.983324,-0.849198,-0.839586,0.839586,-0.270870,0.563230,0.270870,-0.563230,...,-0.976974,-1.114838,-0.507570,-0.499409,-0.236249,0.350938,-0.417514,0.911652,-0.296168,1.073948
4,1101,818,0.143807,-0.617038,-0.924990,0.924990,0.000000,0.990324,0.000000,-0.990324,...,-0.526267,-0.541465,-0.424550,-1.088158,-1.011560,0.614338,0.729495,0.245109,1.526606,2.614378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,459,809,1.529204,1.106682,-1.855327,1.855327,-0.218658,1.450426,0.218658,-1.450426,...,0.509514,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.876984,0.819520,1.320373
1490,1674,887,1.618582,1.752840,0.611392,-0.611392,0.449153,-0.152146,-0.449153,0.152146,...,1.666252,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.932633,-0.085690,0.356356
1491,748,1083,0.856399,0.489199,-0.255778,0.255778,-1.531544,-0.829568,1.531544,0.829568,...,0.358120,0.207905,0.404763,-0.594595,0.894011,0.256338,0.402316,-1.112899,-0.237835,0.067152
1492,1454,1133,0.560689,-0.343777,-0.830239,0.830239,-0.304856,1.210230,0.304856,-1.210230,...,-0.184862,-0.682815,-0.390304,-0.972088,-1.501930,1.215528,1.338708,0.962812,-5.392852,-0.843812


Search for constant feature

In [17]:
non_constants_features = feature_engineering.low_var_features(df_X_train, threshold=0)
print(f"number of features : {df_X_train.shape[1]}")
print(f"number of constant features : {df_X_train.shape[1] - len(non_constants_features)}")

number of features : 34
number of constant features : 0


Search for collinearity or quasi-collinearity

In [63]:
most_correlated_features = feature_engineering.get_most_correlated(df_X_train)
df_collinearity = most_correlated_features.reset_index()
df_collinearity.columns = ['feature 1', 'feature 2', 'correlation']
# df_collinearity.head(30)

set_to_remove_1 = ['DE_NET_IMPORT', 'FR_NET_IMPORT', 'DE_FR_EXCHANGE', 'FR_RESIDUAL_LOAD']
set_to_remove_2 = ['DE_NET_IMPORT', 'FR_NET_IMPORT', 'DE_FR_EXCHANGE', 'FR_RESIDUAL_LOAD', 'DE_WIND', 'DE_CONSUMPTION', 'DE_SOLAR', 'FR_CONSUMPTION']
set_to_remove_3 = ['DE_NET_IMPORT', 'FR_NET_IMPORT', 'DE_FR_EXCHANGE', 'FR_RESIDUAL_LOAD', 'DE_WIND', 'DE_CONSUMPTION', 'DE_SOLAR', 'FR_CONSUMPTION', 'DE_RESIDUAL_LOAD', 'DE_NET_EXPORT', 'DE_LIGNITE']

In [52]:
most_correlated_features_v2 = feature_engineering.get_most_correlated(df_X_train[[col for col in df_X_train.columns if col not in set_to_remove_1]])
col_selection_1 = [col for col in df_X_train.columns if col not in set_to_remove_1]
most_correlated_features_v2.head(15)

DE_WIND         FR_WIND             0.821
DE_CONSUMPTION  FR_CONSUMPTION      0.813
DE_SOLAR        FR_SOLAR            0.803
FR_CONSUMPTION  FR_GAS              0.780
DE_NET_EXPORT   DE_WINDPOW          0.731
DE_COAL         DE_LIGNITE          0.726
DE_GAS          DE_RESIDUAL_LOAD    0.715
DE_LIGNITE      DE_RESIDUAL_LOAD    0.714
DE_WINDPOW      DE_RESIDUAL_LOAD   -0.706
FR_CONSUMPTION  FR_NUCLEAR          0.706
                DE_SOLAR           -0.691
FR_DE_EXCHANGE  DE_NET_EXPORT      -0.686
DE_TEMP         FR_TEMP             0.680
DE_COAL         DE_RESIDUAL_LOAD    0.672
FR_DE_EXCHANGE  FR_NET_EXPORT       0.669
dtype: float64

In [53]:
most_correlated_features_v2 = feature_engineering.get_most_correlated(df_X_train[[col for col in df_X_train.columns if col not in set_to_remove_2]])
col_selection_2 = [col for col in df_X_train.columns if col not in set_to_remove_2]
most_correlated_features_v2.head(15)

DE_NET_EXPORT   DE_WINDPOW          0.731
DE_COAL         DE_LIGNITE          0.726
DE_GAS          DE_RESIDUAL_LOAD    0.715
DE_LIGNITE      DE_RESIDUAL_LOAD    0.714
DE_WINDPOW      DE_RESIDUAL_LOAD   -0.706
FR_DE_EXCHANGE  DE_NET_EXPORT      -0.686
DE_TEMP         FR_TEMP             0.680
DE_COAL         DE_RESIDUAL_LOAD    0.672
FR_DE_EXCHANGE  FR_NET_EXPORT       0.669
DE_NUCLEAR      FR_NUCLEAR          0.659
FR_NUCLEAR      FR_SOLAR           -0.648
DE_WINDPOW      FR_WINDPOW          0.637
DE_GAS          FR_GAS              0.604
DE_COAL         FR_COAL             0.598
FR_GAS          DE_COAL             0.539
dtype: float64

In [64]:
most_correlated_features_v3 = feature_engineering.get_most_correlated(df_X_train[[col for col in df_X_train.columns if col not in set_to_remove_3]])
col_selection_3 = [col for col in df_X_train.columns if col not in set_to_remove_3]
most_correlated_features_v3.head(15)

DE_TEMP         FR_TEMP          0.680
FR_DE_EXCHANGE  FR_NET_EXPORT    0.669
DE_NUCLEAR      FR_NUCLEAR       0.659
FR_NUCLEAR      FR_SOLAR        -0.648
DE_WINDPOW      FR_WINDPOW       0.637
DE_GAS          FR_GAS           0.604
DE_COAL         FR_COAL          0.598
FR_GAS          DE_COAL          0.539
FR_HYDRO        FR_NUCLEAR       0.533
DE_NUCLEAR      FR_SOLAR        -0.487
FR_DE_EXCHANGE  FR_GAS          -0.467
FR_NET_EXPORT   DE_NUCLEAR       0.458
DE_NUCLEAR      FR_WIND         -0.458
FR_GAS          FR_SOLAR        -0.456
FR_NET_EXPORT   FR_GAS          -0.454
dtype: float64

In [67]:
df_X_train_1 = df_X_train.set_index(['ID', 'DAY_ID'])
df_X_train_2 = df_X_train[col_selection_1].set_index(['ID', 'DAY_ID'])
df_X_train_3 = df_X_train[col_selection_2].set_index(['ID', 'DAY_ID'])
df_X_train_4 = df_X_train[col_selection_3].set_index(['ID', 'DAY_ID'])

In [56]:
lr = LinearRegression()

Y_train_clean = df_y_train['TARGET']

lr.fit(df_X_train_1, Y_train_clean)

output_train = lr.predict(df_X_train_1)

def metric_train(output):
    return  spearmanr(output, Y_train_clean).correlation

print('Spearman correlation for the train set: {:.1f}%'.format(100 * metric_train(output_train) ))

Spearman correlation for the train set: 27.6%


In [57]:
lr = LinearRegression()

Y_train_clean = df_y_train['TARGET']

lr.fit(df_X_train_2, Y_train_clean)

output_train = lr.predict(df_X_train_2)

def metric_train(output):
    return  spearmanr(output, Y_train_clean).correlation

print('Spearman correlation for the train set: {:.1f}%'.format(100 * metric_train(output_train) ))

Spearman correlation for the train set: 27.7%


In [58]:
lr = LinearRegression()

Y_train_clean = df_y_train['TARGET']

lr.fit(df_X_train_3, Y_train_clean)

output_train = lr.predict(df_X_train_3)

def metric_train(output):
    return  spearmanr(output, Y_train_clean).correlation

print('Spearman correlation for the train set: {:.1f}%'.format(100 * metric_train(output_train) ))

Spearman correlation for the train set: 27.4%


In [68]:
lr = LinearRegression()

Y_train_clean = df_y_train['TARGET']

lr.fit(df_X_train_4, Y_train_clean)

output_train = lr.predict(df_X_train_4)

def metric_train(output):
    return  spearmanr(output, Y_train_clean).correlation

print('Spearman correlation for the train set: {:.1f}%'.format(100 * metric_train(output_train) ))

Spearman correlation for the train set: 24.2%


We obtain the best spearman's correlation with removing only a few features that have correlation with other variable higher than 0.95

<h3>CROSS-VALIDATION</h3>

In [72]:
from sklearn.model_selection import KFold

k_fold = KFold(n_splits=5, shuffle=True, random_state=6)

for i, (train_index, test_index) in enumerate(k_fold.split(df_X_train_2)):
    print(f"Fold {i}: ")
    print(f"  Train: index={train_index}")
    print(f"  Test: index={test_index}")

Fold 0: 
  Train: index=[   1    3    4 ... 1490 1491 1492]
  Test: index=[   0    2   11   12   14   15   21   24   33   37   40   50   55   56
   58   59   71   73   79   83   88   92   93   98  102  105  109  111
  117  132  136  137  140  148  150  152  155  168  171  178  179  186
  191  195  208  219  222  223  225  229  231  238  245  250  256  257
  261  263  264  268  286  287  290  291  302  304  307  313  329  336
  338  343  357  359  363  371  375  379  381  386  387  399  406  408
  421  430  432  433  434  436  438  439  444  463  465  477  478  486
  488  500  502  503  504  510  511  512  515  517  523  531  553  562
  563  568  570  581  588  592  595  600  602  605  607  608  610  616
  626  628  629  633  646  648  649  682  686  690  702  708  711  718
  733  735  737  739  741  751  752  756  757  761  764  765  767  771
  775  777  785  793  794  796  801  804  811  822  826  830  832  836
  843  851  855  865  874  875  877  883  885  888  894  902  904  905
  9

In [77]:
from sklearn.model_selection import cross_val_score

lr = LinearRegression()

scores = cross_val_score(lr, df_X_train_2, Y_train_clean, cv=k_fold)#, scoring='neg_mean_squared_log_error')
print(scores.mean())
print(scores.std())

0.00599742541935675
0.025925683319626934


Mean squared logarithmic error cannot be used when targets contain negative values. Same goes for Root mean squared logarithmic error